# Abstractive Summarization

This notebook contains a sample for abstractive summarization using chain of density prompting.

In [8]:
!pip install python-dotenv openai

In [9]:
from dotenv import load_dotenv
import logging
import pandas as pd

# Set up logging
logging.basicConfig(level=logging.INFO)

load_dotenv()

True

The OpenAI API is supported by several programs that can serve a local LLM.

| Rank | Program | Description | Supported Model Formats |
|------|---------|-------------|-------------------------|
| 1 | llama.cpp | Highly optimized C++ implementation for running LLMs on consumer hardware. | GGUF, GGML |
| | Website: https://github.com/ggerganov/llama.cpp | Platform Support: Windows, Linux, macOS |
| 2 | text-generation-webui | User-friendly web interface for running various language models locally. | GGUF, GPTQ, AWQ, Transformers |
| | Website: https://github.com/oobabooga/text-generation-webui | Platform Support: Windows, Linux, macOS |
| 3 | Ollama | Simplified tool for running and managing large language models locally. | Custom Ollama format (based on GGUF) |
| | Website: https://ollama.ai | Platform Support: Windows, Linux, macOS |
| 4 | vLLM | High-throughput and memory-efficient inference engine for LLMs. | Transformers, AWQ |
| | Website: https://github.com/vllm-project/vllm | Platform Support: Linux (primary), experimental Windows/macOS |
| 5 | LM Studio | User-friendly GUI application for downloading, running, and chatting with local LLMs. | GGUF |
| | Website: https://lmstudio.ai | Platform Support: Windows, Linux, macOS |
| 6 | ExLlamaV2 | Optimized inference library for running LLMs on consumer GPUs. | EXL2 |
| | Website: https://github.com/turboderp/exllamav2 | Platform Support: Windows, Linux |
| 7 | koboldcpp | Inference engine focused on text generation for creative writing and roleplaying. | GGUF, GGML |
| | Website: https://github.com/LostRuins/koboldcpp | Platform Support: Windows, Linux, macOS |
| 8 | TabbyAPI | Lightweight and efficient API server for running language models locally. | GGUF, EXL2 |
| | Website: https://github.com/theroyallab/tabbyAPI | Platform Support: Windows, Linux, macOS |
| 9 | LiteLLM | Universal API for LLMs, supporting various providers and local models. | Depends on backend (supports multiple) |
| | Website: https://github.com/BerriAI/litellm | Platform Support: Windows, Linux, macOS |
| 10 | llama-cpp-python | Python bindings for llama.cpp, enabling easy integration in Python projects. | GGUF, GGML |
| | Website: https://github.com/abetlen/llama-cpp-python | Platform Support: Windows, Linux, macOS |

Additional notes on model formats:
- GGUF: Successor to GGML, used by many programs for efficient inference
- GGML: Older format, still supported by some tools
- EXL2: Custom format used by ExLlamaV2 (and TabbyAPI) for optimized GPU inference
- AWQ: Activation-aware Weight Quantization, for model compression
- GPTQ: Quantization method for compressing large language models
- Transformers: Format used by the Hugging Face Transformers library
- MLX: Apple's machine learning framework (not widely supported in this list)

In this workflow, I am using TabbyAPI running [command-r](https://huggingface.co/lucyknada/CohereForAI_c4ai-command-r-08-2024-exl2), quanized to 6.0bpw (bits per weight), on an a6000 GPU.

In [3]:
from biagen.llm import OpenAIProvider

llm = OpenAIProvider.from_url("http://127.0.0.1:5000/v1",
                              api_key="e18cbf602915b707311f9601c178e23c",
                              model_name="CohereForAI_c4ai-command-r-08-2024-exl2")

In [4]:
from typing import List

logger = logging.getLogger('abstractive_summary')

def abstractive_summary(article: str, iterations: int = 3) -> List[str]:
    summaries = []
    
    header = f"""You are a copy assistant at an outdoorsman magazine, with a master's degree in journalism. Your task is to provide analysis of the following article with precision.
    
## Article

```article
{article}
```
"""
    
    ## Analysis
    
    prompt_a = f"""{header}
We first must perform an analysis on the article, to identify the most important information.

## Analysis (the most significant topics, with one sentence commentary)

"""

    analysis = llm.generate_one(prompt_a, max_tokens=1024, temperature=0.7, stop_sequences=None)
    alen = len(analysis.split())

    logger.debug(f"analysis {alen} {analysis}")

    # Summarize

    prompt_s = f"""{header}
Given the article and the following analysis, provide a detailed, erudite, succinct, and accurate summary.

## Analysis

{analysis}

## Summary (here is a 5 paragraph summary)

"""

    summary = llm.generate_one(prompt_s, max_tokens=1024, temperature=0.5, stop_sequences=None)

    summaries.append(summary)
    slen = len(summary.split())
    logger.debug(f"summary {slen} {summary}")

    ### Improvement Loop

    for i in range(iterations):
        # Get our last summary

        summary = summaries[-1]

        # Check for missing information in the summary

        prompt_m = f"""{header}
## Current Summary

{summary}

Given the article and the current summary, identify missing information or ways to improve the summary.

## Missing Information (here are 12 entries of novel information not contained in the summary)

"""

        missing = llm.generate_one(prompt_m, max_tokens=1024, temperature=0.7, stop_sequences=None)
        mlen = len(missing.split())
        logger.debug(f"missing {mlen} {missing}")

        # Check for most important information in the summary

        prompt_r = f"""{header}
## Current Summary

{summary}

## Missing Information

{missing}

Using the current summary and the identified missing information, determine if our summary is six (6) paragraphs, identify what information is important and unimportant.

We should be sure to include important facts like size, phone number, unique features, wildlife, history, and contact information.

## Most Important and Interesting Information (here are 12 entries, with one sentence commentary)

"""

        important = llm.generate_one(prompt_r, max_tokens=2048, temperature=0.5, stop_sequences=None)
        ilen = len(important.split())
        logger.debug(f"important {ilen} {important}")

        # Resummarize

        prompt_r = f"""{header}
## Current Summary

{summary}

## Missing Information

{missing}

## Ideas To Consider

{important}

Do not mention the Wildlife Code.

Contact information should be included.

Cover overview, location, features, history, wildlife, game, camping, recreation, contact and management

Do not be brief.

Should be six (6) paragraphs. 

Using the current summary and the identified missing information, here is an improved summary of the article.

## Improved Summary (here is a reorganized and expanded version, including missing information and ideas; six (6) detailed paragraphs)

"""

        resummarized = llm.generate_one(prompt_r, max_tokens=2048, temperature=0.5, stop_sequences=None)
        rlen = len(resummarized.split())
        logger.debug(f"resummarized {rlen} {resummarized}")
        #print(prompt_r, resummarized, rlen)

        summaries.append(resummarized)

    return summaries

In [5]:
# load some data
results_file = 'areas_results.02.jsonl'
details_file = 'areas_details.02.jsonl'

scraped_lakes = pd.read_csv("random_100_areas.tsv", sep='\t')

In [6]:
import os

from biagen.io import append_to_jsonl, read_jsonl

results = read_jsonl(results_file) if os.path.exists(results_file) else []
processed = set(r['area_id'] for r in results)
len(results)

96

In [7]:
model_desc = f"{llm.model}"

for i, row in scraped_lakes.iterrows():
    category = row['category']
    subcategory = row['subcategory']
    area_id = row['area_id']
    area_name = row['area_name']
    article = row['area_info']

    if area_id in processed:
        continue

    summaries = abstractive_summary(article)

    append_to_jsonl({
        'area_id': area_id,
        'area_name': area_name,
        'category': category,
        'subcategory': subcategory,
        'summary': summaries[-1],
        'source': llm.model,
    }, results_file)

    append_to_jsonl({
        'area_id': area_id,
        'area_name': area_name,
        'category': category,
        'subcatgory': subcategory,
        'summary': summaries[-1],
        'summaries': summaries,
        'source': llm.model,
    }, details_file)

    print('=' * 50)
    print(f"{area_name} ({area_id})")
    print('=' * 50)
    for i, s in enumerate(summaries):
        print(f"Round {i}: {s}")
        print('=' * 50)
    print()




INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"


Ruby Clark Willingham Memorial Wildlife Area (ruby-clark-willingham-memorial-wildlife-area)
Round 0: The Ruby Clark Willingham Memorial Wildlife Area, located near Holliday, Missouri, is a 70-acre conservation area with a rich array of wildlife and natural beauty. Open daily from 4:00 AM to 10:00 PM, the area offers a range of activities for visitors, including camping, hiking, sightseeing, and nature observation. Hunting is permitted on the area, with specific regulations in place for deer and turkey hunting during the archery and firearms seasons. Fishing is also allowed on most conservation areas, subject to specific regulations outlined in the Wildlife Code. Vehicle use is restricted to designated roads and parking areas, ensuring the preservation of the natural environment. 

The area's regulations are designed to protect both the wildlife and the natural resources. Target shooting is prohibited to maintain a safe and peaceful environment. Pets and hunting dogs are permitted but m

INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"


The Wayne Helton Memorial Wildlife Area (wayne-helton-memorial-wildlife-area)
Round 0: Nestled in the heart of Missouri, the Wayne Helton Memorial Wildlife Area stands as a testament to conservation and natural beauty. Established in 1969, this expansive 2736-acre haven is dedicated to the memory of J. Wayne Helton, a renowned conservation agent. Within its boundaries lies the Helton Prairie Natural Area, a 30-acre gem boasting an exceptional diversity of native prairie plants, making it a haven for botanists and nature enthusiasts alike. 

The management of the Wayne Helton Memorial Wildlife Area is meticulous, with a range of activities designed to foster a thriving ecosystem. These activities include farming, prescribed burning, and the establishment of native vegetation, all aimed at supporting a wide array of game and non-game wildlife species. The area's regulations are comprehensive, ensuring a harmonious balance between human activities and the preservation of the natural envir

INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"


Vonaventure Memorial Forest and Wildlife Area (vonaventure-memorial-forest-wildlife-area)
Round 0: The Vonaventure Memorial Forest and Wildlife Area, located in Lincoln County, Missouri, holds a rich history. It was generously donated by Henry von Phul Thomas in 1971 with the intent to preserve the natural habitat and forest cover across its 203 acres. The area is easily accessible, situated just north of Silex on Route UU. 

Regulations for activities within the Vonaventure Memorial Forest and Wildlife Area are outlined in the Missouri Code of State Regulations, ensuring the preservation of the area's natural resources and providing a safe environment for visitors. The area is open to the public daily from 4:00 a.m. to 10:00 p.m., with specific activities like hunting, fishing, and dog training permitted 24 hours a day in designated areas. 

Visitors are allowed to engage in various activities such as hiking, sightseeing, nature observation, and wildlife photography. Additionally, the

INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"


KeyboardInterrupt: 